In [1]:
# utilities
import pandas as pd
import numpy as np
from datetime import date

# figure plotting
from bokeh.io import show, curdoc
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, RangeTool, DatetimeTickFormatter,  LabelSet
from bokeh.plotting import figure, show

# widgets
from bokeh.layouts import column, widgetbox
from bokeh.models.widgets import Button, Select, DateRangeSlider

# execute backtest script
import os
import sys
import glob

sys.path.append("../jupyter-py/")
from decode_logs import *
sys.path.pop()

output_dir = "../jupyter-py/output/" + get_current_time()
strategy_type = "kalman"
execution_command = "python ../jupyter-py/backtest_pair.py --strategy_type {} --output_dir {}".format(strategy_type, output_dir)

os.system("rm -rf ../jupyter-py/output")
os.system(execution_command)

0

In [2]:
stock_list = glob.glob("../ib-data/nyse-daily-tech/*.csv")
for i, file in enumerate(stock_list):
    stock_list[i] = os.path.basename(file)[:-4]

In [3]:
data, action_df = Decoder.get_strategy_status(output_dir)
metrics = Decoder.get_strategy_performance(output_dir)

In [4]:
metrics

{'avg_holding_period': 4.666666666666667,
 'endcash': 1177419.7167409933,
 'n_trades': 3,
 'pair': 'AAN-AER',
 'profit': 0.17741971674099333,
 'returnstd': 7361.272303626742,
 'sharperatio': None,
 'startcash': 1000000}

In [129]:
# CODE SECTION: normalized price, figure name = normp_p

# ========== themes & appearance ============= #

STK_1_LINE_COLOR = "#053061"
STK_2_LINE_COLOR = "#67001f"
STK_1_LINE_WIDTH = 1.5
STK_2_LINE_WIDTH = 1.5
WINDOW_SIZE = 10
TITLE = "PRICE OF X vs Y" 
HEIGHT = 250
SLIDER_HEIGHT = 150
WIDTH = 600

# ========== data ============= #
# use sample data from ib-data folder
dates = np.array(data['date'], dtype=np.datetime64)
STK_1_source = ColumnDataSource(data=dict(date=dates, close=data['data0']))
STK_2_source = ColumnDataSource(data=dict(date=dates, close=data['data1']))

# ========== plot data points ============= #
# x_range is the zoom in slider setup. Pls ensure both STK_1 and STK_2 have same length, else some issue
normp = figure(plot_height=HEIGHT, plot_width=WIDTH, x_range=(dates[-WINDOW_SIZE], dates[-1]), title=TITLE, toolbar_location=None)

normp.line('date', 'close', source=STK_1_source, line_color = STK_1_LINE_COLOR, line_width = STK_1_LINE_WIDTH)
normp.line('date', 'close', source=STK_2_source, line_color = STK_2_LINE_COLOR, line_width = STK_2_LINE_WIDTH)
normp.yaxis.axis_label = 'Price'

normp.xaxis[0].formatter = DatetimeTickFormatter()


# ========== RANGE SELECT TOOL ============= #

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=SLIDER_HEIGHT, plot_width=WIDTH, y_range=normp.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=normp.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=STK_1_source, line_color = STK_1_LINE_COLOR, line_width = STK_1_LINE_WIDTH)
select.line('date', 'close', source=STK_2_source, line_color = STK_2_LINE_COLOR, line_width = STK_2_LINE_WIDTH)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

normp_p = column(normp, select)


In [130]:
normp.x_range

Range1d(id='8546', ...)

In [131]:
# CODE SECTION: spread plot, figure name = spread_p
import bokeh.models as bkm
# ========== themes & appearance ============= #

palette = ["#053061", "#67001f"]

LINE_WIDTH = 1.5
LINE_COLOR = palette[-1]
TITLE = "RULE BASED SPREAD TRADING"

# ========== data ============= #

# TODO: get action_source array
# TODO: map actions to colours so can map to palette[i]
# dates = np.array(data['date'], dtype=np.datetime64)
dates = np.array(data['date'], dtype=np.datetime64)
spread_source = ColumnDataSource(data=dict(date=dates, spread=data['spread']))
action_source = ColumnDataSource(action_df)
# action_source['colors'] = [palette[i] x for x in action_source['actions']]


# ========== figure INTERACTION properties ============= #

TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

spread_p = figure(tools=TOOLS, toolbar_location="above", plot_height=HEIGHT, plot_width=WIDTH, title=TITLE)
# spread_p.background_fill_color = "#dddddd"
spread_p.xaxis.axis_label = "Backtest Period"
spread_p.yaxis.axis_label = "Spread"
# spread_p.grid.grid_line_color = "white"


# ========== plot data points ============= #

# plot the POINT coords of the ACTIONS
circles = spread_p.circle("date", "spread", size=12, source=action_source, fill_alpha=0.8)

circles_hover = bkm.HoverTool(renderers=[circles], tooltips = [
    ("Action", "@latest_trade_action"),                    
    ("Stock Bought", "@buy_stk"),
    ("Bought Amount", "@buy_amt"),
    ("Stock Sold", "@sell_stk"),
    ("Sold Amount", "@sell_amt")
    ])

spread_p.add_tools(circles_hover)

# plot the spread over time
spread_p.line('date', 'spread', source=spread_source, line_color = LINE_COLOR, line_width = LINE_WIDTH)
spread_p.xaxis[0].formatter = DatetimeTickFormatter()

# ========== plot label ============= #
# this part you just need to pass BUY or SELL actions
# recommend you use one colour for each action
# x = <col_name_x-axis>, y = <col_name_y-axis> // both from the source dataframe

# labels = LabelSet(x="date", y="spread", text="Prediction", y_offset=8,
#                   text_font_size="8pt", text_color="colors",
#                   source=action_source, text_align='center')

# spread_p.add_layout(labels)

# ========== RANGE SELECT TOOL ============= #
# not included for now because sample data x-axis is not datetime. PLS FIX

# select = figure(title="Drag the middle and edges of the selection box to change the range above",
#                 plot_height=SLIDER_HEIGHT, 
#                 plot_width=WIDTH, 
#                 y_range=spread_p.y_range, 
#                 x_axis_type="datetime", 
#                 y_axis_type=None,
#                 background_fill_color="#efefef")

# range_tool = RangeTool(x_range=spread_p.x_range)
# range_tool.overlay.fill_color = "navy"
# range_tool.overlay.fill_alpha = 0.2

# select.line('date', 'spread', source=spread_source, line_color = LINE_COLOR, line_width = LINE_WIDTH)
# select.ygrid.grid_line_color = None
# select.add_tools(range_tool)
# select.toolbar.active_multi = range_tool

# show(column(spread_p,select))

In [132]:
# CODE SECTION: portfolio value plot, figure name = pv_p

# ========== themes & appearance ============= #

LINE_COLOR = "#053061"
LINE_WIDTH = 1.5
TITLE = "PORTFOLIO VALUE OVER TIME" 

# ========== data ============= #

pv_source = ColumnDataSource(data=dict(date=dates, portfolio_value=data['portfolio_value']))

# ========== plot data points ============= #
# x_range is the zoom in slider setup. Pls ensure both STK_1 and STK_2 have same length, else some issue
pv_p = figure(plot_height=250, plot_width=600, title=TITLE, toolbar_location=None)

pv_p.line('date', 'portfolio_value', source=pv_source, line_color = LINE_COLOR, line_width = LINE_WIDTH)
pv_p.yaxis.axis_label = 'Portfolio Value'

pv_p.xaxis[0].formatter = DatetimeTickFormatter()

In [133]:
# CODE SECTION: setup widgets, widgetbox name = controls_wb

WIDGET_WIDTH = 250

# ========== Select Stocks ============= #
select_stk_1 = Select(width = WIDGET_WIDTH, title='Select Stock 1:', value = stock_list[0], options=stock_list)
select_stk_2 = Select(width = WIDGET_WIDTH, title='Select Stock 2:', value = stock_list[0], options=stock_list)

# ========== Strategy Type ============= #
strategy_list = ['kalman', 'distance', 'cointegration']
select_strategy = Select(width = WIDGET_WIDTH, title='Select Strategy:', value = strategy_list[0], options=strategy_list)

# ========== set start/end date ============= #
# date time variables
MAX_START = date(2014, 1, 1)
MAX_END = date(2018, 12, 30)
DEFAULT_START =date(2016, 1, 1)
DEFAULT_END = date(2018, 1, 30)
STEP = 1

backtest_dates = DateRangeSlider(width = WIDGET_WIDTH, start=MAX_START, end=MAX_END, value=(DEFAULT_START, DEFAULT_END), step=STEP, title="Backtest Date Range:")

start_bt = Button(label="Backtest", button_type="success", width = WIDGET_WIDTH)

# controls = column(select_stk_1, select_stk_2, select_strategy, backtest_dates, start_bt)
controls_wb = widgetbox(select_stk_1, select_stk_2, select_strategy, backtest_dates, start_bt, width=600)


In [135]:
# CODE SECTION: Final layout
grid = gridplot([[controls_wb, normp_p], [pv_p, spread_p]], sizing_mode='fixed')

In [136]:
# CODE SECTION: return curdoc

curdoc().add_root(grid)
# curdoc().title = "DEMO"

In [137]:
show(grid)

In [134]:
# CODE SECTION: setup on_update functions

# this stores the set of params for backtesting
#  params[0] = stk_1, params[1] = stk_2, params[2] = strategy_type, 
# params[3] = start_date, params[4] = end_date 
params = [0, 0, 0, 0, 0] 

# ========== before backtest ============= #
def update_stk_1(attrname, old, new):
    params[0] = select_stk_1.value
    
def update_stk_2(attrname, old, new):
    params[1] = select_stk_2.value
    
def update_strategy(attrname, old, new):
    params[2] = select_strategy.value

def update_dates(attrname, old, new):
    params[3] = backtest_dates.range[0]
    params[4] = backtest_dates.range[1]

select_stk_1.on_change('value', update_stk_1)
select_stk_2.on_change('value', update_stk_2)
select_strategy.on_change('value', update_strategy)
backtest_dates.on_change('range', update_dates)

# ========== backtest ============= #

def run_backtest(new):
    # do something

start_bt.on_click(run_backtest)

IndentationError: expected an indented block (<ipython-input-134-c4296ad340a0>, line 32)

In [11]:
os.getcwd()

'/Users/brendantham/Desktop/FYP/statistical-arbitrage-private-18-19/demo-layout'

In [1]:
import sys

In [2]:
sys.path.insert(0, '../')

In [5]:
from jupyter_py.decode_logs import *

In [6]:
import datetime

In [ ]:
datetime.datetime.fromtimestamp